<a href="https://colab.research.google.com/github/ricardo712/Trabajo/blob/master/Copia_de_Optimization_Problem_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pulp

     |████████████████████████████████| 40.6MB 103kB/s 


In [ ]:
import pulp
from pulp import *
import pandas as pd
import numpy as np

# Data Definitions

In [ ]:
n_warehouses = 2
n_customers = 4
cost_matrix = np.array([[1,3,0.5,4],[2.5,5,1.5,2.5]])

In [ ]:
cust_demands = np.array([35000, 22000, 18000, 30000])

In [ ]:
warehouse_supply = np.array([60000, 80000])

# Model Initialization

In [ ]:
model = LpProblem("Supply-Demand-Problem", LpMinimize)

## Variable Names define

In [ ]:
variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1,n_warehouses+1)]

In [ ]:
variable_names.sort()

In [ ]:
variable_names

['11', '12', '13', '14', '21', '22', '23', '24']

# Decision Variables

In [ ]:
DV_variables = LpVariable.matrix("X", variable_names , cat = "Integer" , lowBound= 0 )

In [ ]:
allocation = np.array(DV_variables).reshape(2,4)

In [ ]:
allocation

array([[X_11, X_12, X_13, X_14],
       [X_21, X_22, X_23, X_24]], dtype=object)

# Objective Function

In [ ]:
lpSum(allocation*cost_matrix)

LpAffineExpression([(X_11, 1.0),
                    (X_12, 3.0),
                    (X_13, 0.5),
                    (X_14, 4.0),
                    (X_21, 2.5),
                    (X_22, 5.0),
                    (X_23, 1.5),
                    (X_24, 2.5)])

In [ ]:
obj_func = lpSum(allocation*cost_matrix)

In [ ]:
# Objective Function 
model +=  obj_func

In [ ]:
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

# Constraints

In [ ]:
# Warehouse Constraints
for i in range(n_warehouses):
    print(lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i] , "Supply Constraints " + str(i)

X_11 + X_12 + X_13 + X_14 <= 60000
X_21 + X_22 + X_23 + X_24 <= 80000


In [ ]:
# Customer Constraints
for j in range(n_customers):
    print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    model += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)

X_11 + X_21 >= 35000
X_12 + X_22 >= 22000
X_13 + X_23 >= 18000
X_14 + X_24 >= 30000


In [ ]:
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
SUBJECT TO
Supply_Constraints_0: X_11 + X_12 + X_13 + X_14 <= 60000

Supply_Constraints_1: X_21 + X_22 + X_23 + X_24 <= 80000

Demand_Constraints_0: X_11 + X_21 >= 35000

Demand_Constraints_1: X_12 + X_22 >= 22000

Demand_Constraints_2: X_13 + X_23 >= 18000

Demand_Constraints_3: X_14 + X_24 >= 30000

VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

# Save problem - Write it

In [ ]:
model.writeLP("Supply_demand_prob.lp")

[X_11, X_12, X_13, X_14, X_21, X_22, X_23, X_24]

# Solve Problem and check Status

In [ ]:
model.solve()
# solvers.PULP_CBC_CMD(fracGap=0)
status =  LpStatus[model.status]
print(status)

Optimal


# Objective Function values and Decision Variable Values


In [ ]:
print("Total Cost:", model.objective.value())
# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Cost: 200000.0
X_11 = 35000.0
X_12 = 22000.0
X_13 = 3000.0
X_14 = 0.0
X_21 = 0.0
X_22 = 0.0
X_23 = 15000.0
X_24 = 30000.0


# Further Analysis

In [ ]:
# Warehouse 1 and Warehouse 2 required capacity
for i in range(n_warehouses):
    print("Warehouse ", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers)))

Warehouse  1
60000.0
Warehouse  2
45000.0
